# Referências da Documentação AWS:

- https://docs.aws.amazon.com/nova/latest/userguide/modalities-document.html

- https://docs.aws.amazon.com/nova/latest/userguide/modalities-document-examples.html

# Dependências

In [ ]:
import os
import json

import boto3

# Constantes

In [ ]:
# Pasta local para salvar respostas das chamadas AWS
OUTPUT_FOLDER = "respostas"

# ID do modelo Amazon Nova Lite (modelo de linguagem da AWS)
MODEL_ID = 'amazon.nova-lite-v1:0'

# Configuração do bucket S3
S3_BUCKET = "NOME_BUCKET"

# Configurações do AWS Bedrock

In [ ]:
# Inicializa o cliente do Bedrock Runtime para fazer chamadas à API
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Método 1: Documento Local (Bytes)

Neste método, lemos um documento do sistema de arquivos local e enviamos seus bytes diretamente para o Bedrock. Este método é ideal para documentos pequenos (até 25 MB) e quando você já tem o arquivo disponível localmente.

In [ ]:
# Ler documento local
DOCUMENT_PATH = "samples/demo.txt"
DOCUMENT_FORMAT = "txt"

print(f"Lendo documento local: {DOCUMENT_PATH}")
with open(DOCUMENT_PATH, "rb") as f:
    doc_bytes = f.read()

# Enviar para Bedrock
messages = [
    {
        "role": "user",
        "content": [
            {
                "document": {
                    "format": DOCUMENT_FORMAT,
                    "name": "documento",
                    "source": {"bytes": doc_bytes}
                }
            },
            {"text": "Faça um resumo executivo deste documento em 3 pontos."}
        ]
    }
]

print("Processando com Bedrock...")
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=messages,
    inferenceConfig={"maxTokens": 500, "temperature": 0.3}
)

# Exibir resultado
resultado = response['output']['message']['content'][0]['text']
print("\nResumo gerado:\n")
print(resultado)
print(f"\nTokens utilizados: {response['usage']['totalTokens']}")

# Método 2: Documento no S3 (s3Location)

Neste método, referenciamos um documento que já está armazenado no Amazon S3. O Bedrock acessa o documento diretamente do S3, sem precisar transferir os bytes. Este método é ideal para documentos grandes (até 2 GB) e quando você quer evitar transferências de dados desnecessárias.

**Importante:** Apenas os modelos Amazon Nova suportam s3Location. Modelos como Claude, Llama e outros não suportam esta funcionalidade.

In [ ]:
# Configuração do documento no S3
S3_KEY = "demo.txt"
DOCUMENT_FORMAT = "txt"

print(f"Usando documento do S3: s3://{S3_BUCKET}/{S3_KEY}")

# Enviar para Bedrock
messages = [
    {
        "role": "user",
        "content": [
            {
                "document": {
                    "format": DOCUMENT_FORMAT,
                    "name": "documento",
                    "source": {
                        "s3Location": {
                            "uri": f"s3://{S3_BUCKET}/{S3_KEY}"
                        }
                    }
                }
            },
            {"text": "Faça um resumo executivo deste documento em 3 pontos."}
        ]
    }
]

print("Processando com Bedrock...")
response = bedrock_runtime_client.converse(
    modelId=MODEL_ID,
    messages=messages,
    inferenceConfig={"maxTokens": 500, "temperature": 0.3}
)

# Exibir resultado
resultado = response['output']['message']['content'][0]['text']
print("\nResumo gerado:\n")
print(resultado)
print(f"\nTokens utilizados: {response['usage']['totalTokens']}")

# Comparação dos Métodos

| Método | Limite | Quando Usar |
|--------|--------|-------------|
| Bytes (local) | 25 MB | Documentos pequenos, já disponíveis localmente |
| S3 Location | 2 GB | Documentos grandes, já armazenados no S3 |


> [Veja documentação oficial da AWS](https://docs.aws.amazon.com/nova/latest/userguide/modalities-document.html)
